<h1> Homework 2 - Data from the Web </h1>

In [208]:
## Importation of everything useful
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import html5lib
sns.set_context('notebook')

import requests
from bs4 import BeautifulSoup

ImportError: No module named 'html5lib'

In [199]:
parameters_page = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')

In [200]:
parameters_html = parameters_page.text

In [201]:
soup = BeautifulSoup(parameters_html, "html.parser")

In [202]:
row_list = []
for parameter_cat_html in soup.find_all('select'):
    for option in parameter_cat_html.contents:
        row_dict = {'category':parameter_cat_html['name'], 'name':option.string, 'value':option['value']}
        row_list.append(row_dict)
parameters = pd.DataFrame(row_list)
parameters.head()

,category,name,value
0,ww_x_UNITE_ACAD,None,null
1,ww_x_UNITE_ACAD,Architecture,942293
2,ww_x_UNITE_ACAD,Chimie et génie chimique,246696
3,ww_x_UNITE_ACAD,Cours de mathématiques spéciales,943282
4,ww_x_UNITE_ACAD,EME (EPFL Middle East),637841336


<h2>Creating the base url</h2>

We found with Postman the base ur
l leading to the students list: http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml

To this page, we will add different parameters. Some of the parameters will not change for all the request, therefore we decided to hardcode them. These parameters are:
<ul>
<li>
<b>ww_x_GPS</b> = <em>-1</em><br>
This parameter varies during request but is not useful. It is probably linked to the gps position. It is was set to -1 which will accept everything.
</li>
<li>
<b>ww_i_reportModel</b> = <em>133685247</em>:<br>
Parameter to choose the type of files to take. We chose html files.
</li>
<li><b>ww_i_reportModelXsl</b> = <em>133685270</em>:<br>
Parameter to choose the type of files to take. We chose html files.
</li>
<li><b>ww_x_HIVERETE</b> = <em>null</em>:<br>
This parameter allow to choose between Winter and Spring semester. However, it is redondant, as Bachelor 1 can only happen in Winter semester, Bachelor 2 in Spring, Bachelor 3 in Winter, and so forth. Therefore we chose to eliminate this parameter by settings its value to null.
</li>
<li><b>ww_x_UNITE_ACAD</b> = <em>249847</em>:<br>
This value represents the "Informatique" section. As it was asked to only consider the data of this section, we have to use it all the time. If it was not the case, we could use the other technique.
</li>


</ul>

In [203]:
people_base_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=null'

***Interesting parameters***<br>
Therefore we are left with two interesting parameters we want to vary:
the year ("periode academique") and the period ("periode pedagogique").

In [204]:
year_param_string = 'ww_x_PERIODE_ACAD'
period_param_string = 'ww_x_PERIODE_PEDAGO'

year_param = parameters.loc[parameters['category'] == year_param_string]
period_param = parameters.loc[parameters['category'] == period_param_string]

year_param = year_param.drop(year_param.iloc[0].name,axis=0, level=None)
period_param = period_param.drop(period_param.iloc[0].name,axis=0, level=None)
period_param = period_param[period_param['name'] != 'Mise à niveau']

In [205]:
year_param.head()

,category,name,value
21,ww_x_PERIODE_ACAD,2016-2017,355925344
22,ww_x_PERIODE_ACAD,2015-2016,213638028
23,ww_x_PERIODE_ACAD,2014-2015,213637922
24,ww_x_PERIODE_ACAD,2013-2014,213637754
25,ww_x_PERIODE_ACAD,2012-2013,123456101


In [206]:
period_param

,category,name,value
32,ww_x_PERIODE_PEDAGO,Bachelor semestre 1,249108
33,ww_x_PERIODE_PEDAGO,Bachelor semestre 2,249114
34,ww_x_PERIODE_PEDAGO,Bachelor semestre 3,942155
35,ww_x_PERIODE_PEDAGO,Bachelor semestre 4,942163
36,ww_x_PERIODE_PEDAGO,Bachelor semestre 5,942120
37,ww_x_PERIODE_PEDAGO,Bachelor semestre 5b,2226768
38,ww_x_PERIODE_PEDAGO,Bachelor semestre 6,942175
39,ww_x_PERIODE_PEDAGO,Bachelor semestre 6b,2226785
40,ww_x_PERIODE_PEDAGO,Master semestre 1,2230106
41,ww_x_PERIODE_PEDAGO,Master semestre 2,942192


*** Make the dataframe and save it on files: ***

In [207]:
def url_param_str(param, value):
    return '&' + param + '=' + value

for i, year in year_param.iterrows():
    for j, period in period_param.iterrows():
            
            url_i = people_base_url + url_param_str(year.category, year.value) + url_param_str(period.category, period.value)
            file_name_i = './data/' + str(year['name']+period['name'])
            
            #print(file_name_i)
            print(url_i)
            try:
                df_i = create_df(url_i)
            except (ValueError, KeyError):
                print('-------> no file here!')
            else:
                df_i.to_pickle(file_name_i)

#test_dataframe = pd.read_pickle("./data/2016-2017Bachelor semestre 1")
#test_dataframe.head()

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=null&ww_x_PERIODE_ACAD=355925344&ww_x_PERIODE_PEDAGO=249108


ImportError: html5lib not found, please install it

## Exercise 1
We import the data stored, and create a list of dataframes for Bachelor 1 and Bachelor 6

In [ ]:
import glob

bachelors_sem1 = glob.glob('.\\data\\[0-9]*-[0-9]*Bachelor semestre 1')
bachelors_sem6 = glob.glob('.\\data\\[0-9]*-[0-9]*Bachelor semestre 6')

sem1_dfs = [pd.read_pickle(data_file) for data_file in bachelors_sem1]
sem6_dfs = [pd.read_pickle(data_file) for data_file in bachelors_sem6]

In [ ]:
sem1_dfs[4].head(1)

We change the academic year according to the fact that the first semester of bachelor start during the first year provided, and the last for Bachelor 6

Ex: for 2012-2013 for Bachelor 1, the correcting_year function will changed it to 2012 and
    for 2014-2015 for Bachelor 6, the correcting_year function will changed it to 2015

In [ ]:
def correcting_year(listOne, listSix):
    #For all Bachelor 1, we select the firt year of the column Period Academic
    for df in listOne:
        df["Period Academic"]=int(df['Period Academic'][2].split('-')[0].replace(" ", ""))

    #For all Bachelor 6, we select the second year of the column Period Academic
    for df in listSix:
        df["Period Academic"] = int(df['Period Academic'][2].split('-')[1])   

In [ ]:
correcting_year(sem1_dfs, sem6_dfs)

In [ ]:
sem1_dfs[4].head(1)

We fisrt concatenate all the dataframes for each semester in a big dataframe
then, we sort them by ascending Period Academic
and finally, for Bachelor 1, we drop duplicate and keep the first entry,
and for Bachelor 6, we drop duplicate and keep the last entry

In [ ]:

bachelors_sem1 = pd.concat(sem1_dfs, ignore_index=True)
bachelors_sem1 = bachelors_sem1.sort_values(['Period Academic'], ascending=True)
bachelors_sem1 = bachelors_sem1.drop_duplicates(subset='No Sciper', keep='first')


bachelors_sem6 = pd.concat(sem6_dfs, ignore_index=True)
bachelors_sem6 = bachelors_sem6.sort_values(['Period Academic'], ascending=True)
bachelors_sem6 = bachelors_sem6.drop_duplicates(subset='No Sciper', keep='last')

bachelors_sem1.shape

In [ ]:
bachelors_sem6.shape

We can observe that the number of student in Bachelor 6 is drastically less than in Bachelor 1, 516 for 1323.

We merge the two big dataframes using the No Sciper

In [ ]:
res = pd.merge(bachelors_sem1, bachelors_sem6, on='No Sciper', how='inner')
res.head(2)

In [ ]:
res.shape

We observed that the number of student that was in Bachelor 1 at the EPFL and in Bachelor 6 are a little bit less than the total number of student in Bachelor 6. We think it is beacause of exchange students ot student that arrived directly in second year (Passerelle).

We create a new column registering the duration of the Bachelor

In [ ]:
final_res = pd.DataFrame({'No Sciper': res['No Sciper'], 'Civilité' :res['Civilité_x'], 'Duration of Bachelor': res['Period Academic_y'] - res['Period Academic_x']})

In [ ]:
#Partition the data between male and female students
grouped_gender = final_res.groupby('Civilité')

In [ ]:
#average duration for female
df_madame = grouped_gender.get_group('Madame')
df_madame.mean()

In [ ]:
#average duration for male
df_monsieur = grouped_gender.get_group('Monsieur')
df_monsieur.mean()

In [ ]:
import scipy

The Wilcoxon rank-sum test tests the null hypothesis that two sets of measurements are drawn from the same distribution. We are testing if male and female durations of bachelor are comming from the same distribution (null hypothesis).

In [ ]:
res = scipy.stats.ranksums( df_madame['Duration of Bachelor'].tolist(), df_monsieur['Duration of Bachelor'].tolist())
res.pvalue

As explained in https://en.wikipedia.org/wiki/P-value - pvalue > 0.1 means that we accept the null hypothesis and so we conclude that there is no significant difference.

## Exerise 2

In [ ]:
masters_sem1 = glob.glob('.\\data\\[0-9]*-[0-9]*Master semestre 1')
masters_sem2 = glob.glob('.\\data\\[0-9]*-[0-9]*Master semestre 2')
masters_sem3 = glob.glob('.\\data\\[0-9]*-[0-9]*Master semestre 3')
projectsA = glob.glob('.\\data\\[0-9]*-[0-9]*Projet Master automne')
projectsP = glob.glob('.\\data\\[0-9]*-[0-9]*Projet Master printemps')

masters_sem1_df_list = [pd.read_pickle(data_file) for data_file in masters_sem1]
masters_sem2_df_list = [pd.read_pickle(data_file) for data_file in masters_sem2]
masters_sem3_df_list = [pd.read_pickle(data_file) for data_file in masters_sem3]
projectsA_df_list = [pd.read_pickle(data_file) for data_file in projectsA]
projectsP_df_list = [pd.read_pickle(data_file) for data_file in projectsP]

In [ ]:
projectsP_df_list[0]

In [ ]:
def correcting_year_Automn(automnDf):
    # We select the second year of the column Period Academic
    for df in automnDf:
        df["Period Academic"]=int(df['Period Academic'][2].split('-')[0].replace(" ", ""))

In [ ]:
correcting_year_Automn(masters_sem1_df_list)
correcting_year_Automn(projectsA_df_list)

In [ ]:
masters_sem1_df_list[0].head(3)

In [ ]:
def correcting_year_Spring(springDf):
    # We select the second year of the column Period Academic
    for df in springDf:
        df["Period Academic"] = int(df['Period Academic'][2].split('-')[1])  

In [ ]:
correcting_year_Spring(masters_sem2_df_list)
correcting_year_Spring(projectsP_df_list)

In [ ]:
masters_sem2_df_list[0].head(3)

We add the master 2 dataframe after the master 1 dataframe (using the append function)

In [ ]:
master1_df = pd.concat(masters_sem1_df_list, ignore_index=True)
master2_df = pd.concat(masters_sem2_df_list, ignore_index=True)
master_courses = master1_df.append(master2_df)
master_courses.head(2)

We sort master 1 and master 2 by date and then by master 2 and master 1 (we put master 2 before master 1 if master 1 and 2 have the same date, since it will mean that the student started its master in spring, so marked as master 2)

In [ ]:
def prepare_sort(row):
    if row['Period pedagogic'] == 'Master semestre 1':
        return -1
    else:
        return -2

master_courses2 = master_courses.copy()
master_courses2['sorting semester'] = master_courses2.apply(prepare_sort, axis=1)
master_courses2.tail(2)
#serie = np.where(master_courses2['Period Academic'] == 'Master semester 1', -1, -2)
#master_courses2['sorting semester'] = {-1 if (master_courses2['Period Academic'] == 'Master semestre 1') else -2}
#master_courses2.head()
#serie

In [ ]:
master_courses_sorted = master_courses2.sort_values(['Period Academic', 'sorting semester'], ascending=True)
master_courses_sorted = master_courses_sorted.drop_duplicates(subset='No Sciper', keep='first')
master_courses_sorted.head(2)